In [1]:
import os
import math
import torch
import numpy as np
import pandas as pd
from torch import nn
from PIL import Image
import torch.nn.functional as F
import torchvision.transforms as T
from torch.autograd import Variable
from timm import create_model, list_models
from torch.utils.data import Dataset, DataLoader

/opt/homebrew/Caskroom/miniforge/base/envs/research/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [85]:
resnet = create_model('resnet18', pretrained=True)
conv1x1 = nn.Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1))

In [258]:
torch.randn(1, 1, 128).expand(-1,10,-1).size()

torch.Size([1, 10, 128])

In [315]:
class Arguments:
    def __init__(self) -> None:
        self.pre_train = False
        self.base_model = 'resnet18'
        self.feature_size = 7
        self.num_cpt = 15
        self.num_classes = 200
        self.cpt_activation = 'att'

args = Arguments()
model = MainModel(args)

In [316]:
a, b, c, d = model(torch.randn(32, 3, 224, 224))

x.shape: torch.Size([32, 49, 128]) x_pe.shape: torch.Size([32, 49, 128])
updates.shape: torch.Size([32, 15, 128]) attn.shape: torch.Size([32, 15, 49])
attn_cls.shape: torch.Size([32, 15])


In [303]:
dots = torch.rand([32, 15, 49])
a = torch.div(dots, dots.sum(2).expand_as(dots.permute([2, 0, 1])).permute([1, 2, 0])) * \
                   dots.sum(2).sum(1).expand_as(dots.permute([1, 2, 0])).permute([2, 0, 1])
b = torch.div(dots, dots.sum(2).unsqueeze(-1).expand_as(dots)) * dots.sum(2).sum(1)[:, None, None].expand_as(dots)
torch.equal(a, b)

True

In [ ]:
dots.sum(2).expand_as(dots.permute([2, 0, 1])).permute([1, 2, 0])) * dots.sum(2).sum(1).expand_as(dots.permute([1, 2, 0])).permute([2, 0, 1]

In [2]:
def pairwise_loss(outputs1, outputs2, label1, label2, sigmoid_param=1.):
    similarity = Variable(torch.mm(label1.data.float(), label2.data.float().t()) > 0).float()
    dot_product = sigmoid_param * torch.mm(outputs1, outputs2.t())
    exp_product = torch.exp(dot_product)

    exp_loss = (torch.log(1 + exp_product) - similarity * dot_product)
    mask_positive = similarity > 0
    mask_negative = similarity <= 0
    S1 = torch.sum(mask_positive.float())
    S0 = torch.sum(mask_negative.float())
    S = S0+S1

    exp_loss[similarity > 0] = exp_loss[similarity > 0] * (S / S1)
    exp_loss[similarity <= 0] = exp_loss[similarity <= 0] * (S / S0)

    loss = torch.mean(exp_loss)

    return loss

In [6]:
batch_size, num_concepts = 32, 10
outputs1 = torch.randn(batch_size, num_concepts)
outputs2 = torch.randn(batch_size, num_concepts)
y1 = torch.randn(batch_size, num_concepts)
y2 = torch.randn(batch_size, num_concepts)


pairwise_loss(outputs1, outputs2, y1, y2)

tensor(2.9113)